In [1]:
ls

ATML-PA-2/


In [2]:
cd ATML-PA-2/Baseline

/kaggle/working/ATML-PA-2/Baseline


In [29]:
%load_ext autoreload
%autoreload 2

In [43]:
import importlib
import utils
import architecture
import train
importlib.reload(utils)
importlib.reload(architecture)
importlib.reload(train)


from utils.dataset import FeatureTensorDataset, OfficeHomeDataset, _truncate_resnet_from
from torchvision.models import ResNet50_Weights
from importlib import reload
import train
from train import train_workflow
import types
from utils.tsne import tsne_plot, get_features
from utils.evaluation import evaluate_ds
import matplotlib.pyplot as plt
import os
from architecture import resnet_classifier

In [31]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [32]:
root_dir = "../datasets/OfficeHomeDataset"
csv_file = f"{root_dir}/ImageInfo.csv"

transform = ResNet50_Weights.IMAGENET1K_V1.transforms()

classes = os.listdir(f"{root_dir}/Art")


In [ ]:
import matplotlib.pyplot as plt

def analysis(source, target, config, cache_source=None, cache_target=None):

    model = resnet_classifier(num_classes=len(classes))

    if cache_source is None:
        source_ds = OfficeHomeDataset(root_dir, csv_file, source, transform)
        cache_source = FeatureTensorDataset(source_ds, model, layer_name='layer3', device=device, batch_size=64)

    if cache_target is None:
        target_ds = OfficeHomeDataset(root_dir, csv_file, target, transform)
        cache_target = FeatureTensorDataset(target_ds, model, layer_name='layer3', device=device, batch_size=64)

    truncated_model = _truncate_resnet_from(model, 'layer3')

    def _forward_impl(self, x):
        f4 = self.layer4(x)
        x = torch.flatten(self.avgpool(f4), 1)
        x = self.fc(x)
        return x

    import types
    truncated_model.forward = types.MethodType(_forward_impl, truncated_model)

    train_workflow(truncated_model, cache_source, cache_target, config, device=device)
    evaluate_ds(cache_target, truncated_model, device, target[0], None)

    latents_s, labels_s = get_features(truncated_model[:-1], cache_source, batchsize=32, device=device)
    latents_t, labels_t = get_features(truncated_model[:-1], cache_target, batchsize=32, device=device)

    if latents_s.dim() > 2:
        latents_s = latents_s.flatten(start_dim=1)
    if latents_t.dim() > 2:
        latents_t = latents_t.flatten(start_dim=1)

    latents = torch.cat([latents_s, latents_t], dim=0)
    domain_labels = torch.cat([
        torch.zeros(latents_s.shape[0], dtype=torch.long),   # 0 = source
        torch.ones(latents_t.shape[0], dtype=torch.long)     # 1 = target
    ])

    fig, ax = plt.subplots(figsize=(8, 6))
    tsne_plot(latents, domain_labels, classes=["Source", "Target"], ax=ax, fig=fig, perplexity=30)
    plt.title("t-SNE Domain Clustering: Source vs Target")
    plt.show()

In [34]:
model = resnet_classifier(num_classes=len(classes))

In [35]:
sources = ["Art", "Real World", "Product"]
target = ["Clipart"]
source_ds = OfficeHomeDataset(root_dir, csv_file, sources, transform)
target_ds = OfficeHomeDataset(root_dir, csv_file, target, transform)

In [ ]:
cache_source = FeatureTensorDataset(source_ds, model, layer_name='layer3', device=device, batch_size=32)
cache_target = FeatureTensorDataset(target_ds, model, layer_name='layer3', device=device, batch_size=32)

Precomputing up to layer3: 100%|██████████| 137/137 [00:47<00:00,  2.91it/s]


In [44]:
config = {'epochs':1, 'lr':1e-3, 'batch_size':32, 'weight_decay':1e-5}
analysis(source=["Art", "Real World", "Product"], target=["Clipart"], config=config, cache_source=cache_source, cache_target=cache_target, )

Training:   0%|          | 0/1 [00:00<?, ?it/s]


Epoch 1/1

Train loss: 1.23387 |  Train acc: 70.39 %

Test loss: 2.37389 | Test acc: 43.41 %



Training: 100%|██████████| 1/1 [00:16<00:00, 16.68s/it]



=== Evaluation for Clipart Domain ===
Accuracy: 43.41%
Confusion Matrix:



TypeError: object of type 'NoneType' has no len()

In [40]:
len(cache_target)

4365